In [1]:
# This cell is included to show what libraries are imported and used in the project
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
import os
import pickle
import math
#import pymc3
import itertools

from glob import glob
from scipy.linalg import expm
import bisect

import tensorflow as tf

%matplotlib inline

/home/tobias/tensorflow_p3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Functions that perform a random transformation (Based on Freifeld article)

# Generate L matrix from eq. 10
def generate_L(N_p):
    rows = N_p - 1
    cols = 2 * N_p
    
    delta = float(1 / N_p)
    
    L = np.zeros((rows, cols))
    
    for i in range(rows):
        L[i][2*i] = (i+1) * delta
        L[i][2*i+1] = 1
        L[i][2*i+2] = -(i+1) * delta
        L[i][2*i+3] = -1
    
    return L


# Find basis of null space of matrix via SVD
def nullspace(A, atol=1e-16, rtol=0):
    """Compute an approximate basis for the nullspace of A.

    The algorithm used by this function is based on the singular value
    decomposition of `A`.

    Parameters
    ----------
    A : ndarray
        A should be at most 2-D.  A 1-D array with length k will be treated
        as a 2-D with shape (1, k)
    atol : float
        The absolute tolerance for a zero singular value.  Singular values
        smaller than `atol` are considered to be zero.
    rtol : float
        The relative tolerance.  Singular values less than rtol*smax are
        considered to be zero, where smax is the largest singular value.

    If both `atol` and `rtol` are positive, the combined tolerance is the
    maximum of the two; that is::
        tol = max(atol, rtol * smax)
    Singular values smaller than `tol` are considered to be zero.

    Return value
    ------------
    ns : ndarray
        If `A` is an array with shape (m, k), then `ns` will be an array
        with shape (k, n), where n is the estimated dimension of the
        nullspace of `A`.  The columns of `ns` are a basis for the
        nullspace; each element in numpy.dot(A, ns) will be approximately
        zero.
    """

    A = np.atleast_2d(A)
    u, s, vh = np.linalg.svd(A)
    tol = max(atol, rtol * s[0])
    nnz = (s >= tol).sum()
    ns = vh[nnz:].conj().T
    return ns

# Psi computation, Eq. 20 in Freifeld
def psi_computation(x, a, b, t):
    if a == 0:
        psi = x + t*b
    else:
        psi = math.exp(t*a)*x + (b*(math.exp(t*a)-1))/a
    
    return psi

# Transformation v1! (Algorithm 1 from Freifeld)
def transformation_v1(P, A, U, N_step, N_p, t=1):
    N_pts = len(U)
    delta_t = float(t) / N_step
    
    phi = np.zeros(N_pts)
    
    for i in range(N_pts):
        phi[i] = U[i]
        
        for j in range(N_step):
            c = bisect.bisect_left(P[1:], phi[i])
            if c == N_p:
                c = c-1
            a = A[2*c]
            b = A[2*c+1]

            phi[i] = psi_computation(phi[i], a, b, delta_t)
        
    return phi


# ------------------------------- Functions for Tensorflow -------------------------------
# Transformation v2! (TENSORFLOW IMPLEMENTATION)
def transformation_v2(A, U, N_step, N_p, t=1):
    delta_t = float(t) / N_step
    
    phi = U
    
    for j in range(N_step):
        
        # Find cell index
        idx = tf.floor(N_p * phi)
        idx = tf.clip_by_value(idx, clip_value_min=0, clip_value_max=N_p-1)
        idx = tf.cast(idx, tf.int32)
        
        # Fetch values from A (vector field)
        a = tf.reshape(tf.gather(A, 2*idx), [-1])
        b = tf.reshape(tf.gather(A, 2*idx+1), [-1])
        
        # Perform psi computation
        phi = tf.where(tf.equal(a, 0), psi_a_eq_zero(phi, a, b, delta_t), psi_a_noteq_zero(phi, a, b, delta_t))
        
    return phi

def psi_a_eq_zero(x, a, b, t):
    tb = tf.multiply(t,b)
    psi = tf.add(x, tb)
    return psi

def psi_a_noteq_zero(x, a, b, t):
    c1 = tf.exp(tf.multiply(t, a))
    c2 = tf.truediv(tf.multiply(b, tf.subtract(c1, 1)), a)
    psi = tf.add(tf.multiply(c1, x), c2)
    return psi

def tf_linear_interpolation(x, x_trans, y, ts_length):
    
    # POSSIBLY RESCALE VALUES IN X_TRANS TO RANGE [0,1] !!!!!!!!!!!!!
    
    # Find nearest smaller neighbor
    dist = tf.subtract(tf.reshape(x_trans, [-1, 1]), x)
    
    # Find index of interval in tessellation
    greater_than_zero = tf.greater_equal(dist, 0)
    idx = (ts_length-1) - tf.reduce_sum(tf.cast(greater_than_zero, tf.float32), axis=0)
    idx = tf.clip_by_value(idx, clip_value_min=0, clip_value_max=ts_length-2)
    idx = tf.cast(idx, tf.int32)
    
    # Fetch values from x_trans and y
    x0 = tf.gather(x_trans, idx)
    x1 = tf.gather(x_trans, idx+1)
    y0 = tf.gather(y, idx)
    y1 = tf.gather(y, idx+1)
    
    # Perform linear interpolation on points in x
    #frac = tf.truediv(tf.subtract(y1, y0), tf.subtract(x1, x0))
    #x_diff = tf.subtract(x, x0)
    #y_interp = tf.add(y0, tf.multiply(x_diff, frac))
    
    y_interp = y0 + (x-x0) * ((y1-y0)/(x1-x0))
    
    return y_interp

In [3]:
PATH = 'UCR_TS_Archive_2015/'
data_sets = {}

In [4]:
ds = 'ProximalPhalanxOutlineAgeGroup'
continue_run = True
skip_transformations = False

In [5]:
ds_list = []
ds_trans_list = []
for folder_PATH in glob('transformations/'+'*'):
    ds_trans = folder_PATH.split("/")[-1]
    ds_split = ds_trans.split("_")[:-1]
    ds_trans_list.append("_".join(ds_split))

In [6]:
for folder_PATH in glob(PATH+'*/'):
    
    print(folder_PATH)
    
    #ds = folder_PATH.split("/")[-2]
    
    if folder_PATH.split("/")[-2] not in ds_trans_list:
        ds_list.append(folder_PATH.split("/")[-2])
    if ds != folder_PATH.split("/")[-2]:
        continue
    data_sets[ds] = {}
    
    print(ds)
    
    with open(folder_PATH + ds + '_TRAIN', 'r') as f:
        
        train = f.read().splitlines()
        data_sets[ds]['TRAIN'] = np.array([train[0].split(",")])
        
        for line in train[1:]:
            data_sets[ds]['TRAIN'] = np.append(data_sets[ds]['TRAIN'], [line.split(",")], axis=0)
            
    with open(folder_PATH + ds + '_TEST', 'r') as f:
        
        test = f.read().splitlines()
        data_sets[ds]['TEST'] = np.array([test[0].split(",")])
        
        for line in test[1:]:
            data_sets[ds]['TEST'] = np.append(data_sets[ds]['TEST'], [line.split(",")], axis=0)

UCR_TS_Archive_2015/SmallKitchenAppliances/
UCR_TS_Archive_2015/Gun_Point/
UCR_TS_Archive_2015/FISH/
UCR_TS_Archive_2015/ElectricDevices/
UCR_TS_Archive_2015/Meat/
UCR_TS_Archive_2015/Car/
UCR_TS_Archive_2015/CinC_ECG_torso/
UCR_TS_Archive_2015/ToeSegmentation2/
UCR_TS_Archive_2015/Cricket_Z/
UCR_TS_Archive_2015/ECG5000/
UCR_TS_Archive_2015/NonInvasiveFatalECG_Thorax1/
UCR_TS_Archive_2015/SonyAIBORobotSurface/
UCR_TS_Archive_2015/uWaveGestureLibrary_Y/
UCR_TS_Archive_2015/SonyAIBORobotSurfaceII/
UCR_TS_Archive_2015/Coffee/
UCR_TS_Archive_2015/Phoneme/
UCR_TS_Archive_2015/Lighting7/
UCR_TS_Archive_2015/UWaveGestureLibraryAll/
UCR_TS_Archive_2015/LargeKitchenAppliances/
UCR_TS_Archive_2015/FordB/
UCR_TS_Archive_2015/Computers/
UCR_TS_Archive_2015/OliveOil/
UCR_TS_Archive_2015/synthetic_control/
UCR_TS_Archive_2015/Ham/
UCR_TS_Archive_2015/ProximalPhalanxOutlineAgeGroup/
ProximalPhalanxOutlineAgeGroup
UCR_TS_Archive_2015/wafer/
UCR_TS_Archive_2015/FordA/
UCR_TS_Archive_2015/MoteStrain/
UC

In [7]:
ds_list

[]

In [8]:
# Set up training and test set
train_size = len(data_sets[ds]['TRAIN'])
test_size = len(data_sets[ds]['TEST'])
ts_length = len(data_sets[ds]['TRAIN'][0])-1

X_train = np.zeros((train_size, ts_length))
y_train = np.zeros(train_size)

X_test = np.zeros((test_size, ts_length))
y_test = np.zeros(test_size)

for i in range(ts_length+1):
    # Train
    for j in range(train_size):
        if i == 0:
            y_train[j] = int(data_sets[ds]['TRAIN'][j][0])
        else:
            X_train[j][i-1] = float(data_sets[ds]['TRAIN'][j][i])
    # Test
    for j in range(test_size):
        if i == 0:
            y_test[j] = int(data_sets[ds]['TEST'][j][0])
        else:
            X_test[j][i-1] = float(data_sets[ds]['TEST'][j][i])
            
if not np.all(y_train):
    zero_idx = True
else:
    zero_idx = False
            
# Make sure the labels are integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Make sure the labels are zero indexed
num_classes = len(np.unique(y_train))

idx = 0
for label in np.unique(y_train):
    y_train[np.where( y_train == label )] = idx
    y_test[np.where( y_test == label )] = idx
    idx += 1
    
# Convert labels to one-hot encoding
y_train_onehot = np.zeros((train_size, num_classes))
y_train_onehot[np.arange(train_size), y_train] = 1

In [9]:
print(y_train)

# Get indices for different classes
class_indices = {}

for label in range(num_classes):
    class_indices[label] = np.where( y_train == label )[0]
    
print(class_indices[0])
print(class_indices[1])

[1 1 1 1 1 1 1 2 2 2 2 2 2 1 1 1 2 1 1 2 2 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 2 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 1 1
 1 1 1 1 1 1 1 2 2 1 1 0 0 1 1 1 2 0 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0
 1 0 1 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 0
 0 0 1 1 0 1 0 1 1 0 0 2 0 1 2 0 1 0 0 1 2 0 0 1 1 0 1 1 1 1 1 0 0 1 1 1 0
 0 1 0 1 0 1 2 1 0 0 0 1 0 0 1 1 1 1 1 2 2 2 0 1 1 2 1 1 1 1 1 1 1 2 0 0 2
 0 1 0 0 1 0 1 0 2 2 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2]
[ 23  24  25  26  27  28  45  46  47  48  60  61  62  63  64  65  70  71
  85  86  91  92 100 101 102 107 108 110 112 115 116 120 121 125 129 130
 137 142 143 147 148 149 152 154 157 158 1

In [10]:
# Get pairs of indices for transformations
transformations_pairs = []

for label in range(num_classes):
    for n in range(train_size):
        for m in range(train_size):
            if n in class_indices[label] and m in class_indices[label]:
                transformations_pairs.append((n,m))
                
# Remove pairs to reduce computations
while (len(transformations_pairs) > 100000 and skip_transformations):
    del transformations_pairs[::4]
    
print(len(transformations_pairs))

60226


In [11]:
# Number of intervals and number of cell intersections
N_p = 10
N_v = N_p + 1

N_step = 100

# Generate tesselation in 1D
tess = np.linspace(0,1,N_v)

# Generate L
L = generate_L(N_p)

# Find basis for null(L)
B = nullspace(L)
[D,d] = B.shape

# Reset graph
tf.reset_default_graph()

In [12]:
-

In [13]:
# Generate transformations for data set

if continue_run:
    with open('transformations/' + ds + '_transformations', 'rb') as f:
        ds_transformations = pickle.load(f)
    idx = len(ds_transformations)
else:
    ds_transformations = np.zeros(d)
    idx = 0

t_pairs = transformations_pairs[idx:]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    idx = 0
    for n,m in t_pairs:
        print('-----' + str(n) + ',' + str(m) + '-----')
             
        feed_dict = {n_idx: n, m_idx: m}
        sess.run(theta.initializer)

        print(sess.run(loss, feed_dict=feed_dict))
        for i in range(100):
            #print(sess.run(loss, feed_dict=feed_dict))
            sess.run(opt, feed_dict=feed_dict)
        print(sess.run(loss, feed_dict=feed_dict))

        # Save transformation parameters
        theta_np = sess.run(theta)

        # Append transformation
        ds_transformations = np.row_stack((ds_transformations,np.transpose(theta_np)[0]))
        
        # Save transformation to file
        if (idx % (1 * len(transformations_pairs) // 100) == 0):
            with open('transformations/' + ds + '_transformations', 'wb') as f:
                pickle.dump(ds_transformations, f)
            print('\n---------------- Saving ' + str(len(ds_transformations)) + ' transformations ----------------\n')
        
        idx += 1

-----370,247-----
0.009777172
0.003071641

---------------- Saving 54785 transformations ----------------

-----370,248-----
0.0044389507
1.026952
-----370,249-----
0.022016065
0.0044672056
-----370,250-----
0.0015326601
0.0012563221
-----370,251-----
0.006001126
1.511707
-----370,252-----
0.0063942424
1.9278647
-----370,253-----
0.002702638
0.0010724443
-----370,254-----
0.0012657281
0.0005582085
-----370,255-----
0.0018669525
0.0007365079
-----370,256-----
0.0021933627
0.002723576
-----370,257-----
0.005831675
0.0020958367
-----370,258-----
0.017173182
0.004842686
-----370,259-----
0.020715853
0.0341371
-----370,260-----
0.00979469
0.0017778802
-----370,261-----
0.0016275051
0.0012133273
-----370,262-----
0.02350472
0.006440227
-----370,263-----
0.010118701
0.003730943
-----370,264-----
0.0019418247
0.0008468152
-----370,265-----
0.003616684
0.0006784264
-----370,266-----
0.004632903
0.001566957
-----370,267-----
0.011685141
0.9751108
-----370,268-----
0.008079429
0.0016225079
-----3

In [14]:
ds_transformations = ds_transformations[1:]
with open('transformations/' + ds + '_transformations', 'wb') as f:
    pickle.dump(ds_transformations, f)

In [15]:
with open('transformations/' + ds + '_transformations', 'rb') as f:
    ds_transformations_pkl = pickle.load(f)
    
print(ds_transformations_pkl)

[[ 5.11476694e-10  1.22460053e-09  1.12128318e-09 ...  2.03458569e-10
   2.43129461e-10  2.59032656e-10]
 [ 1.51093658e-02 -1.52541706e-02 -1.29616335e-02 ...  1.82560273e-02
  -6.75498927e-03  4.84844018e-03]
 [ 1.16100358e-02 -1.71000361e-02 -1.62961539e-02 ...  7.34845363e-03
  -2.83744442e-03 -6.89093163e-03]
 ...
 [-5.31713245e-03  3.08676111e-03 -1.08266668e-02 ... -3.08711641e-02
   8.91997665e-03  9.91108455e-03]
 [-1.51501950e-02  1.18524935e-02 -2.28604581e-02 ... -3.80348898e-02
   1.84677839e-02  2.05197632e-02]
 [ 5.45866630e-10  9.06382303e-10 -5.13718346e-10 ... -1.82982414e-11
   3.09025444e-10  3.32250560e-10]]
